<a href="https://colab.research.google.com/github/vent0906/ww/blob/main/self_learn_GraphSage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GraphSAGE Self-Study Summary

I independently studied GraphSAGE for link prediction using the PyTorch Geometric (PyG) framework. Starting with the Cora citation dataset, I learned to preprocess the graph by splitting edges into training and testing sets, generating negative samples, and formatting them into PyG-compatible structures. I implemented a two-layer GraphSAGE model using mean aggregation, along with both dot product and MLP-based edge predictors. Through training with binary cross-entropy loss and evaluation using AUC scores, I developed a complete pipeline for supervised link prediction on graph data. This project deepened my understanding of graph neural networks, edge-level tasks, and PyG's modular design.


In [6]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.1 MB/s eta 0:00:00


In [2]:
!pip install dgl -f https://data.dgl.ai/wheels/cu118/repo.html

Looking in links: https://data.dgl.ai/wheels/cu118/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 748.2/748.2 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.5 

In [4]:
# 导入相关的库

import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import scipy.sparse as sp


In [7]:

from torch_geometric.datasets import Planetoid

# Load the Cora citation network dataset from the Planetoid benchmark collection.
# This dataset is widely used for semi-supervised node classification in GNN research.
# The dataset will be downloaded and cached in the 'data/Planetoid' directory.
dataset = Planetoid(root='data/Planetoid', name='Cora')

# Access the single graph instance within the Cora dataset.
# Cora contains only one large citation graph.
# The returned object contains node features, edge indices, labels, and train/val/test masks.
data = dataset[0]



Processing...
Done!


In [9]:
from torch_geometric.utils import to_scipy_sparse_matrix
# Extract edge index and convert to numpy arrays (u, v pairs)
edge_index = data.edge_index
u = edge_index[0].numpy()
v = edge_index[1].numpy()

# Generate shuffled edge IDs
eids = np.arange(edge_index.size(1))
np.random.shuffle(eids)

# Split positive edges into training and testing sets (90/10 split)
test_size = int(len(eids) * 0.1)
train_size = len(eids) - test_size
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

# Build adjacency matrix to find non-existent (negative) edges
adj = to_scipy_sparse_matrix(edge_index, num_nodes=data.num_nodes).tocoo()
adj_dense = adj.toarray()

# Generate the negative edge candidates: 1 - A - I
adj_neg = 1 - adj_dense - np.eye(data.num_nodes)
neg_u, neg_v = np.where(adj_neg > 0)

# Randomly sample the same number of negative edges as positive ones
neg_eids = np.random.choice(len(neg_u), len(eids), replace=False)
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]

# Convert all edge sets to PyTorch edge_index format (shape [2, num_edges])
train_pos_edge_index = torch.tensor([train_pos_u, train_pos_v], dtype=torch.long)
train_neg_edge_index = torch.tensor([train_neg_u, train_neg_v], dtype=torch.long)
test_pos_edge_index = torch.tensor([test_pos_u, test_pos_v], dtype=torch.long)
test_neg_edge_index = torch.tensor([test_neg_u, test_neg_v], dtype=torch.long)


In [13]:
# Step 1: Get the complete edge index from the original graph
edge_index = data.edge_index  # shape: [2, num_edges]

# Step 2: Generate a shuffled list of edge indices
eids = np.arange(edge_index.size(1))
np.random.shuffle(eids)

# Step 3: Split into test edges (10%) and train edges (90%)
test_size = int(len(eids) * 0.1)
train_edges = eids[test_size:]  # 90% for training
test_edges = eids[:test_size]   # 10% for testing

# Step 4: Create a new edge_index tensor containing only the training edges
train_edge_index = edge_index[:, train_edges]

# Step 5: Deep-copy the original graph and replace edge_index with training-only edges
train_data = deepcopy(data)
train_data.edge_index = torch.tensor(train_edge_index, dtype=torch.long)

# Now `train_data` is a graph with test edges removed,
# which can be safely used for training the GNN without label leakage

<ipython-input-13-30f97188ebfb>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_data.edge_index = torch.tensor(train_edge_index, dtype=torch.long)


In [14]:
from torch_geometric.nn import SAGEConv

# Define a 2-layer GraphSAGE model using PyTorch Geometric
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, hidden_feats):
        super(GraphSAGE, self).__init__()
        # First GraphSAGE layer: input -> hidden dimension
        self.conv1 = SAGEConv(in_feats, hidden_feats, aggr='mean')  # mean aggregation
        # Second GraphSAGE layer: hidden -> hidden dimension
        self.conv2 = SAGEConv(hidden_feats, hidden_feats, aggr='mean')

    def forward(self, x, edge_index):
        """
        Forward pass of GraphSAGE model.

        Args:
            x (Tensor): Node feature matrix of shape [num_nodes, in_feats]
            edge_index (LongTensor): Edge list in COO format [2, num_edges]

        Returns:
            Tensor: Node embeddings of shape [num_nodes, hidden_feats]
        """
        h = self.conv1(x, edge_index)  # Apply first SAGEConv
        h = F.relu(h)                  # Apply non-linearity
        h = self.conv2(h, edge_index)  # Apply second SAGEConv
        return h  # Output final node embeddings

In [15]:

# Convert training positive and negative edge pairs into edge_index tensors.
# Each edge_index is of shape [2, num_edges] in PyG.

# Training positive edges: (u, v) → [2, num_train_pos_edges]
train_pos_edge_index = torch.tensor([train_pos_u, train_pos_v], dtype=torch.long)

# Training negative edges: (u, v) → [2, num_train_neg_edges]
train_neg_edge_index = torch.tensor([train_neg_u, train_neg_v], dtype=torch.long)

# Testing positive edges: used for AUC evaluation
test_pos_edge_index = torch.tensor([test_pos_u, test_pos_v], dtype=torch.long)

# Testing negative edges: used for AUC evaluation
test_neg_edge_index = torch.tensor([test_neg_u, test_neg_v], dtype=torch.long)

<ipython-input-15-d080c9faff8e>:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  train_pos_edge_index = torch.tensor([train_pos_u, train_pos_v], dtype=torch.long)


In [16]:
# Dot Product Predictor: scores edges using the dot product of node embeddings
class DotPredictor(nn.Module):
    def forward(self, h, edge_index):
        """
        Args:
            h (Tensor): Node embeddings of shape [num_nodes, hidden_dim]
            edge_index (LongTensor): Edge list of shape [2, num_edges]

        Returns:
            Tensor: Edge scores of shape [num_edges], higher means more likely to exist
        """
        src, dst = edge_index
        # Element-wise product and sum → dot product
        score = (h[src] * h[dst]).sum(dim=1)
        return score


# MLP-based Predictor: scores edges using a learned 2-layer feedforward network
class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def forward(self, h, edge_index):
        """
        Args:
            h (Tensor): Node embeddings of shape [num_nodes, hidden_dim]
            edge_index (LongTensor): Edge list of shape [2, num_edges]

        Returns:
            Tensor: Edge scores of shape [num_edges]
        """
        src, dst = edge_index
        edge_input = torch.cat([h[src], h[dst]], dim=1)  # concat source and destination
        x = F.relu(self.W1(edge_input))
        x = self.W2(x)
        return x.squeeze(1)  # shape: [num_edges]



In [17]:
# Initialize GraphSAGE model and predictor
model = GraphSAGE(data.num_features, 16)  # GraphSAGE: input_dim → hidden_dim
pred = DotPredictor()  # You can also use MLPPredictor(16)

# Compute binary cross-entropy loss from positive and negative edge scores
def compute_loss(pos_score, neg_score):
    """
    Compute binary classification loss for link prediction.

    Args:
        pos_score (Tensor): Scores for positive (real) edges
        neg_score (Tensor): Scores for negative (fake) edges

    Returns:
        Tensor: Binary cross-entropy loss
    """
    scores = torch.cat([pos_score, neg_score], dim=0)
    labels = torch.cat([
        torch.ones(pos_score.size(0), device=pos_score.device),
        torch.zeros(neg_score.size(0), device=neg_score.device)
    ])
    return F.binary_cross_entropy_with_logits(scores, labels)

# Compute AUC score for evaluation
def compute_auc(pos_score, neg_score):
    """
    Compute ROC AUC score for link prediction.

    Args:
        pos_score (Tensor): Scores for positive edges
        neg_score (Tensor): Scores for negative edges

    Returns:
        float: AUC score
    """
    scores = torch.cat([pos_score, neg_score], dim=0).detach().cpu().numpy()
    labels = torch.cat([
        torch.ones(pos_score.size(0)),
        torch.zeros(neg_score.size(0))
    ]).cpu().numpy()
    return roc_auc_score(labels, scores)




In [18]:

# 1. Initialize optimizer to jointly update the GNN model and the edge predictor
# Using Adam optimizer with a learning rate of 0.01
optimizer = torch.optim.Adam(
    itertools.chain(model.parameters(), pred.parameters()),
    lr=0.01
)

# 2. Training loop for link prediction
for epoch in range(100):
    model.train()  # Set model to training mode
    pred.train()   # Set predictor (Dot or MLP) to training mode

    # Step 1: Forward pass through GNN to compute node embeddings
    h = model(data.x, data.edge_index)  # data.x: node features, data.edge_index: graph structure

    # Step 2: Compute predicted scores for positive and negative training edges
    pos_score = pred(h, train_pos_edge_index)  # Scores for edges that exist
    neg_score = pred(h, train_neg_edge_index)  # Scores for sampled non-existing edges

    # Step 3: Compute binary classification loss (1 for pos edges, 0 for neg edges)
    loss = compute_loss(pos_score, neg_score)

    # Step 4: Backward pass and optimizer update
    optimizer.zero_grad()  # Clear previous gradients
    loss.backward()        # Compute gradients
    optimizer.step()       # Update parameters

    # Optional: Print loss every 5 epochs
    if epoch % 5 == 0:
        print(f"Epoch {epoch:03d} | Loss: {loss.item():.4f}")

# 3. Evaluation phase — compute AUC on the test set
model.eval()  # Set GNN model to evaluation mode
pred.eval()   # Set predictor to evaluation mode

with torch.no_grad():  # Disable gradient tracking for evaluation
    h = model(data.x, data.edge_index)  # Compute node embeddings again

    # Predict link scores for test set edges
    pos_score = pred(h, test_pos_edge_index)
    neg_score = pred(h, test_neg_edge_index)

    # Compute ROC AUC score: how well the model separates positive from negative links
    auc = compute_auc(pos_score, neg_score)
    print(f"Final Test AUC: {auc:.4f}")

Epoch 000 | Loss: 0.7122
Epoch 005 | Loss: 0.6723
Epoch 010 | Loss: 0.5882
Epoch 015 | Loss: 0.5398
Epoch 020 | Loss: 0.5177
Epoch 025 | Loss: 0.4903
Epoch 030 | Loss: 0.4700
Epoch 035 | Loss: 0.4499
Epoch 040 | Loss: 0.4269
Epoch 045 | Loss: 0.4055
Epoch 050 | Loss: 0.3854
Epoch 055 | Loss: 0.3651
Epoch 060 | Loss: 0.3457
Epoch 065 | Loss: 0.3263
Epoch 070 | Loss: 0.3056
Epoch 075 | Loss: 0.2833
Epoch 080 | Loss: 0.2596
Epoch 085 | Loss: 0.2345
Epoch 090 | Loss: 0.2085
Epoch 095 | Loss: 0.1853
Final Test AUC: 0.8513
